In [3]:
import pickle
import os
import re
from google_trans_new import google_translator
from httpcore import SyncHTTPProxy 
import copy
import nltk
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
import string
import enchant
import locale
import sys
from langdetect import detect
from langdetect import DetectorFactory
import datetime
import demoji
demoji.download_codes()
from textblob import TextBlob

C:\Users\david\anaconda3\envs\thesis\lib\site-packages\ipykernel_launcher.py:18: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.


In [4]:
import Topic_modeling

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cm
%matplotlib inline
plt.rcParams["figure.figsize"] = (30,30)

import pandas as pd
import numpy as np

from gensim.models import Word2Vec
import gensim, logging


from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from sklearn.feature_extraction.text import TfidfVectorizer

OMP_NUM_THREADS=1

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\david\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
locale.setlocale(locale.LC_ALL, 'en_US')

'en_US'

In [4]:
# txt file with the list of fb pages/group
with open('fb_pages_id_list.txt') as fb:
    id_fb_list = fb.read().splitlines() 

In [5]:
#scraper
def scrape_fb(group_id, credentials = ('garrybage20@gmail.com' ,'piocane123'), timeout = 80, pages = 300, posts_per_page = 500):
    facebok_scraper = list()
    from facebook_scraper import get_posts
    for post in get_posts(group_id, credentials, 
                               timeout, pages, posts_per_page):
                        #cookie = 'C:\\Users\\david\\OneDrive\\Documents\\thesis\\facebook-scraper-master\\facebook_scraper\\cookies.txt'):
        facebok_scraper.append(post)
    return facebook_scraper

In [6]:
#scrape every fb pages
def scrape_these_fb(pages_list):
    all_pages = []
    for page in id_fb_list:
        all_pages.append(scrape_fb(page))
    return all_pages

In [7]:
def no_nested_list_anymore(nested_list):
    final_scraped_data = []
    for chunk in nested_list:
        for item in chunk:
            final_scraped_data.append(item)
    return final_scraped_data

In [8]:
#feature filtering
def filter_dictionary(dataset, keys = ('post_id','text','time','user_id')):
    dict_filter = lambda x, y: dict([(i,x[i]) for i in x if i in set(y)])
    new_dict = []
    for post in dataset:
        post = dict_filter(post,keys)
        new_dict.append(post)
    return new_dict

In [9]:
#nontype text cleaning
def remove_none_type(dataset):
    new_dataset = []
    for post in dataset:
        if post['text'] is not None:
            new_dataset.append(post)
    return new_dataset

In [10]:
# strip preprocessing that helps for fb post
def double_strip(dataset):
    new_dataset = copy.deepcopy(dataset)
    for post in new_dataset:
        for key,value in post.items():
            if key == 'text':
                if value is None:
                    pass
                else:
                    post[key] = re.sub(r'(?<!\n)\n(?!\n)|\n{2,}', ' ', value)
                    #post[key] = value.split()
    return new_dataset

In [11]:
#emoji detection
def emoji_scan(dataset):
    new_dataset = copy.deepcopy(dataset)
    for post in new_dataset:
        post['emoji'] = demoji.findall(post['text'])
    return new_dataset

In [12]:
#external_url detection 
def external_url_scan(dataset):
    new_dataset = copy.deepcopy(dataset)
    regex = r'('
    # Scheme (HTTP, HTTPS, FTP and SFTP):
    regex += r'(?:(https?|s?ftp):\/\/)?'
    # www:
    regex += r'(?:www\.)?'
    regex += r'('
    # Host and domain (including ccSLD):
    regex += r'(?:(?:[A-Z0-9][A-Z0-9-]{0,61}[A-Z0-9]\.)+)'
    # TLD:
    regex += r'([A-Z]{2,6})'
    # IP Address:
    regex += r'|(?:\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})'
    regex += r')'
    # Port:
    regex += r'(?::(\d{1,5}))?'
    # Query path:
    regex += r'(?:(\/\S+)*)'
    regex += r')'
    for post in new_dataset:
            post['external_url'] = []
            find_urls_in_string = re.compile(regex, re.IGNORECASE)
            url_collector = []
            for site in find_urls_in_string.findall(post['text']):
                url_collector.append(site[0])
            if url_collector != []:
                for url in url_collector:
                    post['external_url'].append(url)
                    post['text'] = post['text'].replace('{}'.format(url), '')
            else:
                post['external_url'] = []
            post['external_url'] = list(set(post['external_url']))
    return new_dataset

In [13]:
#language scan 
DetectorFactory.seed = 0
def language_scan(dataset):
    new_dataset = copy.deepcopy(dataset)
    count = 0 
    for post in new_dataset:
            try:
                if post['text'] is None:
                    pass
                else:
                    post['text'] = post['text'].lower()
                    post['text_language'] = detect(post['text'])
            except:
                language = 'error'
            count += 1
            sys.stdout.write("\r {}/{} post scanned ...".format(count,len(new_dataset)))
            sys.stdout.flush()
        
    return new_dataset

In [14]:
# languages count in a dataset 
def languages_count(dataset):
    language_list = []
    for post in range(len(dataset)):
        language_list.append(dataset[post]['text_language'])
        
    count_lang = dict()
    for language in set(language_list):
        count_lang[language] = 0
    
    for language in count_lang:
        for item in dataset:
            if item['text_language'] == language:
                count_lang[language] += 1
    return count_lang

In [15]:
# remove post duplicates from a dataset 
def remove_duplicate(dataset):
    seen = set()
    new_data = []
    for post in dataset:
        if post['text'].lower().replace(' ','') not in seen:
            seen.add(post['text'].lower().replace(' ',''))
            new_data.append(post)
    return new_data

In [16]:
#remove non textual post (run it after language scan)
def remove_non_text(dataset):
    new_dataset = []
    for post in dataset:
        if 'text_language' in post:
            new_dataset.append(post)
    return new_dataset

In [17]:
# translate the post in english if necessary
def translate(dataset):
    new_dataset = copy.deepcopy(dataset)
    #proxy = {'http' : '193.8.56.119:9183:kzeamash:ic9v35spcdzb'}
    proxy = {'http' : '114.115.211.214:8085'}
    translator = google_translator(proxies=proxy)
    count = 0 
    for entries in new_dataset:
        for key, value in entries.items():
            if key == 'text':
                if value is None:
                    pass
                else:
                    entries[key] = translator.translate(value)
        count += 1
        sys.stdout.write("\r {}/{} post scanned ...".format(count,len(new_dataset)))
        sys.stdout.flush()
    return new_dataset

In [18]:
# give a key that inform whether one post shows a phone contact or not 
def has_phone(dataset):
    new_dataset = copy.deepcopy(dataset)
    for post in new_dataset:
        if re.findall("(?:\+\d{1,3}|0\d{1,3}|00\d{1,2})?(?:\s?\(\d+\))?(?:[-\/\s.]|\d)+", post['text']):
            for find in re.findall("(?:\+\d{1,3}|0\d{1,3}|00\d{1,2})?(?:\s?\(\d+\))?(?:[-\/\s.]|\d)+", post['text']):
                if len(find)>10:
                    post['phone_number'] = 'yes'
                    post['text'] = post['text'].replace('{}'.format(find), '')
                else:
                    post['phone_number'] = 'no'
    return new_dataset

In [19]:
#salary detection 
# quantity
def add_regex_digit_in(a_list):
    '''Have more '''
    output_list = []
    for word in a_list:
        output_list.append(word + "\d+")
        output_list.append('\d+'+ word) 
    return output_list

def add_regex_for_salary_in(a_list):
    output_list = []
    for word in a_list:
        output_list.append(word + "\d+\,\d+")
        output_list.append("\d+\,\d+" + word)
        output_list.append(word + "\d+\.\d+")
        output_list.append("\d+\.\d+" + word)
    return output_list
# unit
# official salary list
def salary_check(dataset):
    '''identify whether a salary is proposed in the job offer and extract the unit in order to calculate it'''

    first = ['e', 'eur','euro','euros', 'h','hour','hours','net', 'netto', 'gross', 'gross salary', 'gross tariff', 'net salary', 'net tariff', 'egross', 'crude']
    middle = ['/', 'per']
    last = ['month','week','hour','w','h']
    solo_bonus = ['weekly','wage','salary','bonus','plus','additives','extra']    #'e'
    solo_word = [ 'eur','euro','euros','hour','hours','net', 'netto', 'gross', 'gross salary', 'gross tariff', 'net salary', 'net tariff', 'egross']
    
    unit_combo = []
    for one in first:
        '''create a list for unit salary terms like 'currency/time' '''
        for two in middle:
            for three in last:
                unit_combo.append(one + two + three)
    weak_combo = []           
    for two in middle:
        '''create a list for unit salary terms like '/time' ''' 
        for three in last:
            weak_combo.append(two + three)
            
    new_dataset = copy.deepcopy(dataset)
        
        
    salary_signals = ['€\d+', '\d+€', 'eur\d+', '\d+eur', '\d+\$', '\$\d+']
    salary_signals_full = ['\€','eur','euro','euros','\$']
    
    for post in new_dataset: 
        '''identify salary and salary unit for each list created above'''
        post['salary'] = None
        for signal in list(set(salary_signals + add_regex_digit_in(solo_bonus) + \
                               add_regex_digit_in(solo_word) + add_regex_digit_in(unit_combo) + add_regex_digit_in(weak_combo))):
            if re.search('{}'.format(signal), post['text'].replace(' ','').lower()):
                if post['salary'] is None:
                    post['salary'] = re.findall('{}'.format(signal), post['text'].replace(' ','').lower())
                else:
                    post['salary'].extend(re.findall('{}'.format(signal), post['text'].replace(' ','').lower()))
                for signals in add_regex_for_salary_in(salary_signals_full):
                    #print(add_regex_for_salary_in(salary_signals_full))
                    if re.search('{}'.format(signals), post['text'].replace(' ','').lower()):
                        post['salary'] = re.findall('{}'.format(signals), post['text'].replace(' ','').lower())
                    else:
                        pass
            elif post['salary'] != None:
                pass                
            else:
                post['salary'] = None
        if post['salary'] != None:
            post['salary'] = list(set(post['salary']))      
    return new_dataset

def salary_info(dataset):
    new_dataset = copy.deepcopy(dataset)
    for post in new_dataset:
        post['salary_info'] = None
        if post['salary'] is not None:
            for item in post['salary']:
                if post['salary_info'] is None and re.findall('[a-zA-Z]', item) != '':
                    temp = re.findall('[a-zA-Z]', item)
                    temp = ''.join(temp)
                    post['salary_info'] = [temp]
                elif re.findall('[a-zA-Z]', item) != '':
                    temp = ''.join(re.findall('[a-zA-Z]', item))
                    post['salary_info'].append(temp)
    return new_dataset

def float_salary(dataset):
    new_dataset = copy.deepcopy(dataset)
    for post in new_dataset:
        for key,value in post.items():
            float_value = []
            post['salary'] = list(filter(None, post['salary']))
            for item in range(len(post['salary'])):
                float_value.append(float(post['salary'][item]))
            post['salary'] = float_value
    return new_dataset

def salary_final(dataset):
    new_dataset = copy.deepcopy(dataset)
    for post in range(len(new_dataset)):
        if new_dataset[post]['salary'] is not None:
            for item in range(len(new_dataset[post]['salary'])):
                new_dataset[post]['salary'][item] = re.sub('[^0-9,.]', '', new_dataset[post]['salary'][item])
                new_dataset[post]['salary'][item] = new_dataset[post]['salary'][item].replace(',','.')
    return new_dataset           

def take_salary(dataset):
    new_dataset = []
    for post in all_data_salary_good:
        if post['salary'] != None:
            new_dataset.append(post)
    return new_dataset

In [20]:
#after salary check
def remove_meaningless_data(dataset):
    new_dataset = []
    for post in dataset:
        if len(post['text'])>150 or (len(post['text'])<150 and post['salary'] is not None):
            new_dataset.append(post)
    return new_dataset

In [21]:
def salary_hour(dataset):
    new_dataset = copy.deepcopy(dataset)
    for post in new_dataset:
        post['salary'] = max(post['salary'])
        if post['salary'] > 50 and post['salary'] < 1000:
            post['salary'] = post['salary']/40
        if post['salary'] >= 1000:
            post['salary'] = post['salary']/173
    for post in new_dataset:
        if 'gross' in post['text'].lower() or 'cross' in post['text'].lower() or 'crude' in post['text'].lower():
            pass
        elif 'net' in post['text'].lower():
            for element in net_salary:
                if str(post['time'].year) == element[1] and int(post['salary']) == element[2]:
                    post['salary'] = element[0]
        else:
            pass
    return new_dataset

In [22]:
def spelling_correction(dataset):
    new_dataset = copy.deepcopy(dataset)
    count = 0
    for post in new_dataset:
        temp_text = TextBlob(post['text'])
        textCorrected = temp_text.correct()
        post['text'] = textCorrected
        count += 1
        sys.stdout.write("\r {}/{} post scanned ...".format(count,len(dataset)))
        sys.stdout.flush()
    return new_dataset

In [23]:
def compare(text1, text2):  
    '''compare the spelling corrected text with the original one'''
    l1 = text1.split()
    l2 = text2.split()
    good = 0
    bad = 0
    for i in range(0, len(l1)):
        if l1[i] != l2[i]:
            bad += 1
        else:
            good += 1
    return (good, bad)

# Helper function to calculate the percentage of misspelled words
def percentageOfBad(x):
    return (x[1] / (x[0] + x[1])) * 100

In [24]:
# manually change something
def add_salary_from(dataset, post_id, salary):
    '''manually update the salary in the dataset'''
    for post in dataset: #all_data_float:
        if post['post_id'] == post_id:
            post['salary'] = [salary]
    return dataset

In [25]:
words = set(nltk.corpus.words.words())

def cleaned_txt(dataset):
    new_dataset = copy.deepcopy(dataset)
    for post in new_dataset:
        post['text'] = post['text'].lower()
        post['text'] = " ".join(i for i in nltk.wordpunct_tokenize(post['text']) if i.isalpha())
        post['text'] = re.sub(r'\s*\d+\s*', '', post['text'])

    return new_dataset

In [370]:
def remove_more_duplication(dataset):
    corpus = dataset['text']
    count_vect = CountVectorizer()
    X_train_counts = count_vect.fit_transform(corpus)
    cos_df = pd.DataFrame(cosine_similarity(X_train_counts))
    i, j = np.indices(cos_df.shape).reshape(2, -1)
    cos_values = cos_df.values.reshape(-1)
    cos_sim_df = pd.DataFrame({'i': i, 'j': j, 'sim':cos_values})
    cos_rem = cos_sim_df[(cos_sim_df['sim']>0.90)&(i!=j)]
    cos_rem['i*j'] = cos_rem['i'] * cos_rem['j']
    drop_rows = np.unique(cos_rem.drop_duplicates(subset=['i*j'], keep='first')['i'].values)
    dataset = dataset[~dataset.index.isin(drop_rows)]
    dataset = dataset.reset_index()
    dataset.drop(columns={'index'}, inplace=True)
    return dataset

# Work in Progress Script

In [ ]:
#how to get the data from all the fb sources in one variable
no_nested_list_anymore(scrape_these_fb(id_fb_list))

In [ ]:
with open(r"C:\Users\david\OneDrive\Documents\thesis\facebook-scraper-master\storage+\Jobs in The Netherlands.pickle", 'rb') as d:
    data_one = pickle.load(d)

In [ ]:
with open(r"C:\Users\david\OneDrive\Documents\thesis\facebook-scraper-master\storage+\AgroJobsHolland.pickle", 'rb') as da:
    data_two = pickle.load(da)

In [ ]:
#do not keep it 
from os import listdir
from os.path import isfile, join
real_path = 'C:\\Users\\david\\OneDrive\\Documents\\thesis\\facebook-scraper-master\\storage+\\'
onlyfiles = [f for f in listdir(r'C:\Users\david\OneDrive\Documents\thesis\facebook-scraper-master\storage+') if isfile(join(r'C:\Users\david\OneDrive\Documents\thesis\facebook-scraper-master\storage+', f))]

def loadall(path, files):
    data = []
    for file in files:
        with open(path + file, "rb") as f:
            while True:
                try:
                    data.append(pickle.load(f))
                except EOFError:
                    break
    return data

In [ ]:
all_data = loadall(real_path,onlyfiles)

In [ ]:
all_data = no_nested_list_anymore(all_data)

In [ ]:
all_data = remove_none_type(all_data)

In [ ]:
all_data = double_strip(all_data)

In [ ]:
min_dict_keys = ('post_id','text','time','user_id')

In [ ]:
all_data_simple = filter_dictionary(all_data,min_dict_keys)

In [ ]:
all_data_text_scan = remove_non_text(all_data_lan_scan)

In [ ]:
all_data_trans = translate(all_data_text_scan

In [ ]:
all_data_trans = no_nested_list_anymore(all_data_trans)

In [ ]:
all_data_emo = emoji_scan(all_data_trans)

In [ ]:
all_data_url = external_url_scan(all_data_emo)

In [ ]:
all_data_phone = has_phone(all_data_url)

In [ ]:
all_data_single = remove_duplicate(all_data_phone)

In [ ]:
all_data_url_fixed = external_url_scan(all_data_single)

In [ ]:
all_data_phone_fixed = has_phone(all_data_url_fixed)

In [ ]:
all_data_s = salary_check(all_data_phone_fixed)

In [ ]:
all_data_r = remove_meaningless_data(all_data_s)

In [ ]:
all_data_c = spelling_correction(all_data_r)

In [281]:
all_data_info = salary_info(all_data_soc)

In [282]:
all_data_salary_good = salary_final(all_data_info) 

In [283]:
all_data_filter = take_salary(all_data_salary_good) 

In [284]:
all_data_float = float_salary(all_data_filter)

In [285]:
tuple_update_salary = [('686536082198629',14.00),('1482600698551552',11.00),('717440619153453',12.00)]#, ('2361088324168782',11.00)]     

In [286]:
for item in tuple_update_salary:
    add_salary_from(all_data_float, item[0],item[1])

[{'post_id': '997070214376256',
  'text': '‼️we are looking for reachtruck, forklift-and pallet carrier drivers‼️ €10,55 -€11,20 gross/hour requirements: ▪️knowledge of english language is considered an advantage ▪️driving license cat. b ▪️motivate,also flexible and ability to work in a team ▪️availability (minimum of 3 months) we offer: ✔️weekly salary payment ✔️transport local ✔️nice work at different companies ✔️full time job offers ✔️long term cooperation ✔️accommodation according to sn standard are you interested in our offers? apply by filling in the form on our webster  are you polish or ukrainian than you can contact our office by phone are you rumania than you can contact our recruits by phone #agrojobsholland #growwithus ',
  'time': datetime.datetime(2021, 7, 5, 12, 0, 57),
  'user_id': '178700066213279',
  'text_language': 'en',
  'emoji': {'✔️': 'check mark',
   '▪️': 'black small square',
   '‼️': 'double exclamation mark'},
  'external_url': ['www.agro.jobs/register/'],


[{'post_id': '997070214376256',
  'text': '‼️we are looking for reachtruck, forklift-and pallet carrier drivers‼️ €10,55 -€11,20 gross/hour requirements: ▪️knowledge of english language is considered an advantage ▪️driving license cat. b ▪️motivate,also flexible and ability to work in a team ▪️availability (minimum of 3 months) we offer: ✔️weekly salary payment ✔️transport local ✔️nice work at different companies ✔️full time job offers ✔️long term cooperation ✔️accommodation according to sn standard are you interested in our offers? apply by filling in the form on our webster  are you polish or ukrainian than you can contact our office by phone are you rumania than you can contact our recruits by phone #agrojobsholland #growwithus ',
  'time': datetime.datetime(2021, 7, 5, 12, 0, 57),
  'user_id': '178700066213279',
  'text_language': 'en',
  'emoji': {'✔️': 'check mark',
   '▪️': 'black small square',
   '‼️': 'double exclamation mark'},
  'external_url': ['www.agro.jobs/register/'],


[{'post_id': '997070214376256',
  'text': '‼️we are looking for reachtruck, forklift-and pallet carrier drivers‼️ €10,55 -€11,20 gross/hour requirements: ▪️knowledge of english language is considered an advantage ▪️driving license cat. b ▪️motivate,also flexible and ability to work in a team ▪️availability (minimum of 3 months) we offer: ✔️weekly salary payment ✔️transport local ✔️nice work at different companies ✔️full time job offers ✔️long term cooperation ✔️accommodation according to sn standard are you interested in our offers? apply by filling in the form on our webster  are you polish or ukrainian than you can contact our office by phone are you rumania than you can contact our recruits by phone #agrojobsholland #growwithus ',
  'time': datetime.datetime(2021, 7, 5, 12, 0, 57),
  'user_id': '178700066213279',
  'text_language': 'en',
  'emoji': {'✔️': 'check mark',
   '▪️': 'black small square',
   '‼️': 'double exclamation mark'},
  'external_url': ['www.agro.jobs/register/'],


In [287]:
with open(r"new_data.txt") as d:
    add_data = list()
    for i in d.readlines():
        add_data.append((i.strip().split(',')[0],float(i.strip().split(',')[1])))
    new_data = []
    for new in add_data:
        for post in all_data_info:
            if post['post_id'] == new[0]:
                post['salary'] = [new[1]]
                new_data.append(post)
                all_data_float.append(post)

In [288]:
all_data_update = []
for post in all_data_float:
    if post['salary'] is not None and post['salary'] != []:
        all_data_update.append(post)

In [289]:
for post in all_data_update:
    if post['salary'] is None or post['salary'] == []:
        print(post)

In [290]:
with open(r'update_salary.txt') as di:
    fixed_data = list()
    for ele in di.readlines():
        fixed_data.append((ele.strip().split(',')[0],float(ele.strip().split(',')[1])))
    for fixed in fixed_data:
        for post in all_data_update:
            if post['post_id'] == fixed[0]:
                post['salary'] = [fixed[1]]

In [292]:
salary_check = list()
for post in data_salary:
    salary_check.append((post['post_id'],max(post['salary']),post['text']))

salary_check.sort(key=lambda x: x[1])

In [293]:
with open('net_salary.pickle', 'rb') as net_:
    net_salary = pickle.load(net_)

In [294]:
final_data = salary_hour(data_salary)

In [355]:
data_full =  add_data_no_salary +final_data

In [356]:
data_cleaned = cleaned_txt(data_full)

In [358]:
data_df = pd.DataFrame(data_cleaned)

In [371]:
df = remove_more_duplication(data_df)

C:\Users\david\anaconda3\envs\thesis\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [373]:
final_d = df.to_dict('records')

In [374]:
with open('job_description_keywords.txt') as f:
    keywords = f.read().splitlines() 

In [1331]:
#do some manual transformation
def a_mali_estremi(dataset):
    new_dataset = copy.deepcopy(dataset)
    for post in new_dataset:
        post['text'] = post['text'].lower()
        if 'mob' in post['text'] and 'automobile' not in post['text'] and 'mobile' not in post['text']:
            post['text'] = post['text'].replace('mob','job')
        if 'wicker' in post['text']:
            post['text'] = post['text'].replace('wicker','picker')
        if 'fires' in post['text']:
            post['text'] = post['text'].replace('fires','fries')
        if 'rickets' in post['text']:
            post['text'] = post['text'].replace('rickets','picker')
        if 'klaudiaaccommodation' in post['text']:
            post['text'] = post['text'].replace('klaudiaaccommodation','accommodation')
        if 'accommodationwijk' in post['text']:
            post['text'] = post['text'].replace('accommodationwijk','accommodation')
        if 'klaudiakasiaofficeaccommodation' in post['text']:
            post['text'] = post['text'].replace('klaudiakasiaofficeaccommodation','accommodation') 
        if 'juliaklaudiaofficeaccommodation' in post['text']:
            post['text'] = post['text'].replace('juliaklaudiaofficeaccommodation','accommodation') 
        if 'kasiaofficewe' in post['text']:
            post['text'] = post['text'].replace('kasiaofficewe','office')
        if 'coeur' in post['text']:
            post['text'] = post['text'].replace('coeur','eur')
    return new_dataset

In [1332]:
for_topic = []
for post in a_mali_estremi(final_d):
    if post['post_id'] not in po:
        for_topic.append(post)

In [1319]:
with open(r"C:\Users\david\Desktop\nl-town-list\town.txt") as f:
    lines = f.readlines()
    lines = [i.strip() for i in lines]
    del lines[1]
    town_list = [name.lstrip() for item in lines for name in item.split(',')]

In [382]:
for_topic_t = copy.deepcopy(for_topic)
for post in for_topic_t:
    for town in town_list:
        town = town.lower()
        post['text'] = post['text'].lower().replace(town,'')

In [1334]:
len(for_topic_t)

2925

In [1322]:
len(temp_lista)

2925

In [1339]:
final_for_topic

[{'post_id': '987301022019842',
  'text': 't you branch for sharing this nice update of your time with us it is a pleasure to have you on our team come join our team for the  job offers in agriculture production and logistics we offeraltro housing transport and support on a peral level during your stay register now on our webster and our recruited will contact you agrojobsh growwithus appreciationtuesday agrochamps',
  'time': Timestamp('2021-06-15 12:07:42'),
  'user_id': '178700066213279',
  'text_language': 'en',
  'emoji': {},
  'external_url': ['www.agro.jobs/register'],
  'phone_number': 'yes',
  'salary': None,
  'salary_info': None,
  'job_type': 'agriculture,forestry and fishing'},
 {'post_id': '986723368744274',
  'text': 'go sobs h can h you with the  jobs good accommodation transport and contact in a peral level register today on our website sobs at potatoes for couples register now for jobs in the netherlands register',
  'time': Timestamp('2021-06-14 15:38:48'),
  'user_i

In [1066]:
with open('final_for_topic.pickle', 'wb') as d:
    pickle.dump(final_for_topic,d)

In [57]:
with open('final_for_topic.pickle', 'rb') as d:
    final_for_topic = pickle.load(d)

In [58]:
#df = pd.DataFrame(cleaned_txt(final_for_topic))

In [59]:
real_one = copy.deepcopy(final_for_topic)
for post in real_one:
    if post['external_url'] != []:
        post['external_url'] = 'yes'
    else:
        post['external_url'] = 'no'

In [61]:
for i in real_one:
    i.pop('user_id')
    i.pop('salary_info')
    i.pop('post_id')
    i.pop('emoji') 

In [62]:
for i in real_one:
    i['time'] = i['time'].year
  # print(i['time'].year)

In [64]:
df = pd.DataFrame(real_one)